#### S1 - 스피어 피싱 대화 데이터셋 생성기

이 스크립트는 **OpenAI API**를 사용해 **스피어 피싱(개인정보 보호자 타깃) 대화 데이터셋**을 생성합니다.  
총 **289개의 시나리오**를 기반으로, 각 시나리오당 1~2개의 대화를 만들어 **약 500개 데이터**를 생성합니다.

---

✅ 주요 기능
- **자연스러운 한국인 이름 자동 생성**
  - `KoreanNameGenerator`를 이용해 중복 없는 자연스러운 이름을 자동 생성
- **100개 단위로 분할 저장**
  - 500개 데이터를 5개 JSON 파일로 나누어 저장
- **API 실 호출 기반 생성**
  - 실제 OpenAI API 호출로 데이터 생성
  - 총 토큰 사용량 출력

---

⚙️ 실행 순서
1. API 키 입력 → OpenAI 클라이언트 초기화  
2. 시나리오 목록 로드 (289개)  
3. 이름 자동 생성 (중복 방지)  
4. 대화 생성 (시나리오별 1~2개)  
5. 100개 단위로 JSON 파일 저장  
6. 총 토큰 사용량 출력  

In [1]:
import os
import json
import time
import random
from typing import Dict, List, Optional, Set
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
from datetime import datetime

1. 설정 및 상수

In [ ]:
# API 설정
OPENAI_MODEL = 'gpt-4o'
API_DELAY_SECONDS = 1.0

# 생성 설정
TOTAL_TARGET_SAMPLES = 500  # 총 생성할 데이터 수
BATCH_SIZE = 100  # 배치당 데이터 수
NUM_BATCHES = 5  # 총 배치 수

# 출력 설정
OUTPUT_DIR = "s1_generated_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

2. 한국인 이름 생성기

In [ ]:
import random
from typing import List, Set

class KoreanNameGenerator:
    """중복 없는 자연스러운 한국인 이름 생성기 (성과 이름 첫 글자 중복 방지)"""

    # 한국 성씨
    SURNAMES = [
        "김", "이", "박", "최", "정", "강", "조", "윤", "장", "임",
        "한", "오", "서", "신", "권", "황", "안", "송", "전", "홍",
        "유", "고", "문", "양", "손", "배", "백", "허", "남", "심",
        "노", "하", "곽", "성", "차", "주", "우", "구", "민", "류"
    ]

    # 이름 200개
    GIVEN_NAMES = [
        "민석", "서준", "도윤", "하준", "주원", "시우", "지호", "지후", "유준", "현우",
        "하윤", "준우", "예준", "건우", "승우", "민재", "연우", "준호", "시윤", "현준",
        "예진", "하연", "유주", "소윤", "유진", "민서", "지유", "세은", "주하", "다현",
        "주호", "지훈", "준혁", "하민", "도훈", "지헌", "하성", "유찬", "민호", "승호",
        "서연", "하은", "지우", "서윤", "지민", "수아", "하린", "예린", "채원", "유나",
        "지원", "지윤", "다은", "수빈", "아린", "하율", "예은", "서하", "우빈", "성현",
        "도현", "시현", "태현", "민성", "승민", "지안", "현민", "나은", "가은", "하영",
        "시연", "서우", "예원", "민하", "채윤", "예서", "승현", "진우", "은우", "태윤",
        "현서", "우진", "재윤", "성민", "재원", "지환", "지현", "소율", "하늘", "은서",
        "서희", "예나", "수연", "서린", "유리", "라윤", "지한", "도연", "태민", "하진",
        "윤호", "하람", "지혁", "민결", "은찬", "유담", "준영", "재민", "하온", "윤서",
        "서진", "은호", "민우", "재현", "예담", "지완", "유성", "세윤", "나윤", "다온",
        "서빈", "도하", "현진", "윤재", "가율", "하빈", "은재", "민규", "유건", "시훈",
        "태율", "민솔", "도겸", "유하", "하경", "주은", "태린", "지율", "현이", "민채",
        "서현", "라희", "하엘", "지영", "은별", "도아", "예슬", "나리", "수현", "가빈",
        "세영", "연진", "나율", "지효", "라온", "민교", "서유", "예솔", "주리", "윤아",
        "은유", "유민", "도유", "하음", "시온", "유현", "예아", "연서", "하예", "재하",
        "도율", "하랑", "서담", "지유", "라미", "나현", "가현", "채린", "예온", "시안",
        "도진", "지혁", "은지", "민호", "유찬", "하율", "나윤", "가온", "도은", "예린"
    ]

    def __init__(self):
        self.used_names: Set[str] = set()
        self.generation_count = 0

    def generate_name(self, max_attempts: int = 100) -> str:
        """중복 없는 한국인 이름 생성 (성과 이름 첫 글자 중복 방지)"""
        for _ in range(max_attempts):
            surname = random.choice(self.SURNAMES)

            # 이름 후보 중에서 성의 첫 글자와 다르게 시작하는 이름만 필터링
            valid_given_names = [name for name in self.GIVEN_NAMES if not name.startswith(surname)]
            if not valid_given_names:
                valid_given_names = self.GIVEN_NAMES  # 혹시 전부 제외될 경우 대비

            given_name = random.choice(valid_given_names)
            full_name = surname + given_name

            if full_name not in self.used_names:
                self.used_names.add(full_name)
                self.generation_count += 1
                return full_name

        # 중복 시 숫자 추가 (fallback)
        surname = random.choice(self.SURNAMES)
        given_name = random.choice(self.GIVEN_NAMES)
        full_name = f"{surname}{given_name}{self.generation_count}"
        self.used_names.add(full_name)
        self.generation_count += 1
        return full_name

    def generate_names(self, count: int) -> List[str]:
        """여러 개의 중복 없는 이름 생성"""
        return [self.generate_name() for _ in range(count)]

    def reset(self):
        """사용된 이름 초기화"""
        self.used_names.clear()
        self.generation_count = 0


# 전역 이름 생성기 초기화
name_generator = KoreanNameGenerator()

print("✅ 한국인 이름 생성기 초기화 완료")
print("\n테스트 생성 이름 10개:")
for i, name in enumerate(name_generator.generate_names(10), 1):
    print(f"  {i}. {name}")

✅ 한국인 이름 생성기 초기화 완료

테스트 생성 이름 10개:
  1. 박하율
  2. 강수빈
  3. 오우진
  4. 양우진
  5. 정서준
  6. 문시윤
  7. 강시현
  8. 이채윤
  9. 류현준
  10. 고수아


3. 토큰 추적기

In [4]:
class TokenTracker:
    """토큰 사용량 추적 클래스"""
    
    def __init__(self):
        self.total_prompt_tokens = 0
        self.total_completion_tokens = 0
        self.total_tokens = 0
        self.request_count = 0
        self.batch_stats = []  # 배치별 통계
    
    def add_usage(self, usage):
        """API 응답에서 토큰 사용량 추가"""
        self.total_prompt_tokens += usage.prompt_tokens
        self.total_completion_tokens += usage.completion_tokens
        self.total_tokens += usage.total_tokens
        self.request_count += 1
    
    def add_batch_stats(self, batch_num: int, samples: int, tokens: int):
        """배치 통계 추가"""
        self.batch_stats.append({
            'batch': batch_num,
            'samples': samples,
            'tokens': tokens
        })
    
    def get_summary(self):
        """사용량 요약 반환"""
        return {
            'total_requests': self.request_count,
            'prompt_tokens': self.total_prompt_tokens,
            'completion_tokens': self.total_completion_tokens,
            'total_tokens': self.total_tokens,
            'avg_tokens_per_request': round(self.total_tokens / self.request_count, 2) if self.request_count > 0 else 0
        }
    
    def print_summary(self):
        """사용량 요약 출력"""
        summary = self.get_summary()
        print("\n" + "="*80)
        print("📊 토큰 사용량 요약")
        print("="*80)
        print(f"총 API 요청 수: {summary['total_requests']:,}회")
        print(f"\n토큰 사용량:")
        print(f"  • 입력 토큰:    {summary['prompt_tokens']:,}")
        print(f"  • 출력 토큰:    {summary['completion_tokens']:,}")
        print(f"  • 총 토큰:      {summary['total_tokens']:,}")
        print(f"  • 평균/요청:    {summary['avg_tokens_per_request']:,.2f}")
        
        if self.batch_stats:
            print(f"\n배치별 통계:")
            for stat in self.batch_stats:
                print(f"  배치 {stat['batch']:2d}: {stat['samples']:3d}개 샘플, {stat['tokens']:,} 토큰")
        
        print("="*80)


# 전역 토큰 추적기
token_tracker = TokenTracker()
print("✅ 토큰 추적기 초기화 완료")

✅ 토큰 추적기 초기화 완료


4. 시나리오 로드

In [5]:
# 시나리오 로드
with open('S1_scenarios.json', 'r', encoding='utf-8') as f:
    ALL_SCENARIOS = json.load(f)

print(f"✅ 총 {len(ALL_SCENARIOS)}개의 시나리오를 로드했습니다.")
print(f"\n샘플 시나리오 (처음 3개):")
for i, s in enumerate(ALL_SCENARIOS[:3]):
    print(f"  {i+1}. {s['type']} - {s['method']} - {s['approach']}")

✅ 총 289개의 시나리오를 로드했습니다.

샘플 시나리오 (처음 3개):
  1. 급여 계좌 확인 요청 - HR 담당자 사칭 - 급여 이체 오류로 인한 긴급 계좌 재확인 명목
  2. 외부 법무 조사 - 법무법인 변호사 사칭 - 계약 관련 소송 대비 자료 요청 명목
  3. 수표/계좌 입금 확인 - 금융기관 콜센터 사칭 - 비정상 거래 확인 명목


5. 시스템 프롬프트

In [6]:
SYSTEM_INSTRUCTION = """당신은 보안 연구용 스피어 피싱(개인정보 보호자 타깃) 대화 데이터 생성 전문가입니다.

**목적**: IT/보안 전문가를 대상으로 하는 정교한 사회공학 공격 대화를 생성합니다.

**출력 형식**: 반드시 유효한 JSON만 출력하세요.
```json
{
  "dialogue": "발화1\\n발화2\\n발화3...",
  "label": 1,
  "meta": {
    "attack_type": "spear_phishing",
    "scenario_tag": "success_attack",
    "attack_outcome": "success",
    "target_role": "보안팀 주임",
    "domain": "보안운영",
    "attack_vector": "phone",
    "first_sensitive_spoken": "10 23 45 67",
    "fake_values": {"ip": "10 23 45 67", "port": "8080"},
    "confidence": "high",
    "psychological_tactics": ["권위", "긴박감", "전문용어"],
    "victim_response": "협조적"
  }
}
```

**핵심 규칙**:
1. **label = 1 (피싱 공격)**
2. 공격 성공률 70% (30%는 피해자가 의심하고 거부)
3. 모든 민감정보는 허구값 (IP: 10.x.x.x, 192.168.x.x만)
4. STT 형식: 소문자, 특수문자 없음, 발화는 \\n으로 구분

5. **IT 전문가 피해자 특성**:
   - 간투사: "음", "어", "그"
   - 전문 용어 사용: "서버가", "로그 확인했는데", "포트 번호가"
   - 신중한 확인: "음 그럼 티켓 번호가 어떻게 되시죠", "어 부서 어디신가요"
   - 절차 언급: "이메일로 먼저 받아야 할 것 같은데", "승인 절차가"
   - 기술적 질문: "어떤 CVE 번호인가요", "패치 버전은"

6. **공격자 전략 (정교한 사회공학)**:
   **초반 - 신뢰 구축 (3-4턴)**:
   - 내부 용어 사용: "티켓 시스템이", "젠킨스 빌드가"
   - 구체적 프로젝트명/시스템명 언급
   - 공동 지인/상사 이름 드롭: "김 팀장님께서 말씀하시길"
   - 침착하고 전문적인 톤

   **중반 - 긴박감 조성 (3-6턴)**:
   - 기술적 긴급성: "지금 프로덕션 서버에 문제가", "롤백 필요한 상황"
   - 시간 압박: "30분 내 패치해야", "고객사 SLA 위반될 수 있어서"
   - 책임 암시: "혹시 문제 생기면", "로그 남으니까"
   - 목소리 톤 약간 긴장

   **후반 - 정보 획득 (4-7턴)**:
   - 단계적 요청: 먼저 비민감 정보 → 점차 민감 정보로
   - 기술적 정당화: "SSH 접속 확인만", "읽기 권한만"
   - 협력 유도: "같이 확인해보시죠", "화면 공유 어려우면"

7. **통화 환경 디테일**:
   - 피해자 상황: "지금 배포 중이긴 한데", "모니터링하고 있어서"
   - 기술적 배경음: "알람 울리고 있는데", "회의 들어가야 해서"
   - 멀티태스킹: "자료 찾아보면서 말씀드릴게요", "로그 확인 중입니다"

8. **전문가 특유의 대화 패턴**:
   - 신원/권한 확인: "음 죄송한데 사번이 어떻게 되시죠", "어느 부서신가요"
   - 절차 준수: "먼저 티켓 등록하시고", "이메일로 요청 주시면"
   - 기술적 검증: "어떤 서버 말씀이신가요", "IP 대역이 어떻게"
   - 재확인: "제가 다시 정리하면", "그럼 요약하자면"

9. **감정 변화 (피해자)**:
   - 성공 시: 경계 → 바쁨/압박 → 협조 → 정보 제공
   - 실패 시: 경계 → 의심 강화 → 절차 고수 → 확고한 거부

10. **공격자 말투 변화**:
   - 초반: 전문적이고 협력적
   - 중반: 긴박하고 권위적
   - 후반: 설득적 또는 압박적 (상황에 따라)

11. 대화 길이: 10-17턴
12. 플레이스홀더 절대 금지
13. 실존 기관명 금지 (가상의 회사명만)
14. 화자 표시 없이 발화만 출력
"""

print("✅ 시스템 프롬프트 로드 완료")

✅ 시스템 프롬프트 로드 완료


6. OpenAI 클라이언트 초기화

In [7]:
def initialize_client() -> Optional[OpenAI]:
    """OpenAI 클라이언트 초기화 (input으로 키 입력)"""
    
    # 환경변수에 없으면 직접 입력 요청
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        api_key = input("🔑 OpenAI API 키를 입력하세요: ").strip()

    if not api_key:
        print("❌ API 키가 입력되지 않았습니다.")
        return None

    try:
        client = OpenAI(api_key=api_key)
        print("✅ OpenAI 클라이언트 초기화 완료")
        return client
    except Exception as e:
        print(f"❌ 클라이언트 초기화 실패: {e}")
        return None


# 실행 시 입력 요청됨
client = initialize_client()

✅ OpenAI 클라이언트 초기화 완료


7. 대화 생성 함수

In [8]:
def generate_single_conversation(
    client: OpenAI,
    scenario: Dict,
    participant_names: List[str]
) -> Optional[Dict]:
    """단일 대화 생성"""
    
    # 공격 성공/실패 결정 (70% 성공)
    attack_should_succeed = random.random() < 0.70
    
    # 피해자 역할 선택
    target_roles = [
        "보안팀 주임",
        "시스템 관리자",
        "개발팀 리더",
        "DB 관리자",
        "네트워크 엔지니어",
        "보안 담당자",
        "인프라 운영팀",
        "데이터 보안 담당자"
    ]
    target_role = random.choice(target_roles)
    
    # 피해자 상황 설정
    victim_situations = [
        {"context": "배포 작업 중", "state": "바쁨"},
        {"context": "모니터링 중", "state": "집중"},
        {"context": "장애 대응 중", "state": "긴장"},
        {"context": "회의 직전", "state": "급함"},
        {"context": "점심시간", "state": "여유"},
        {"context": "문서 작업 중", "state": "집중"},
        {"context": "테스트 중", "state": "바쁨"}
    ]
    victim_situation = random.choice(victim_situations)
    
    # 공격자 심리 전술
    tactics = [
        ["권위", "긴박감", "전문용어"],
        ["신뢰", "협력", "시간압박"],
        ["전문성", "긴급성", "책임암시"],
        ["동료의식", "기술용어", "압박"]
    ]
    selected_tactics = random.choice(tactics)
    
    outcome_text = "성공" if attack_should_succeed else "실패"
    
    user_prompt = f"""
시나리오: {scenario['type']}
공격 방법: {scenario['method']}
목표 정보: {scenario['target']}
접근 방식: {scenario['approach']}

대화 참여자 이름: {', '.join(participant_names)}

**피해자 설정**:
- 역할: {target_role}
- 상황: {victim_situation['context']}
- 상태: {victim_situation['state']}

**공격자 전술**: {', '.join(selected_tactics)}

**중요: 공격 결과는 "{outcome_text}"로 설정할 것**
- attack_outcome을 반드시 "{'success' if attack_should_succeed else 'failed'}"로 설정하세요.

위 시나리오에 맞는 자연스러운 스피어 피싱 대화를 생성하세요.
대화에 이름을 직접 언급하지 말고, meta 정보의 participants에만 포함하세요.
"""
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTION},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.9,
            response_format={"type": "json_object"}
        )
        
        # 토큰 사용량 추적
        token_tracker.add_usage(response.usage)
        
        # 응답 파싱
        content = response.choices[0].message.content
        data = json.loads(content)
        
        # 메타 정보에 참여자 추가
        if 'meta' not in data:
            data['meta'] = {}
        data['meta']['participants'] = participant_names
        
        # 시나리오 정보 추가
        data['scenario_type'] = scenario['type']
        data['scenario_method'] = scenario['method']
        data['scenario_approach'] = scenario['approach']
        
        return data
        
    except json.JSONDecodeError as e:
        print(f"  ⚠️  JSON 파싱 실패: {e}")
        return None
    except Exception as e:
        print(f"  ⚠️  생성 실패: {e}")
        return None


print("✅ 대화 생성 함수 정의 완료")

✅ 대화 생성 함수 정의 완료


8. 배치 생성 함수

In [9]:
def generate_batch(
    client: OpenAI,
    batch_num: int,
    scenarios: List[Dict],
    target_samples: int,
    delay: float = 1.0
) -> pd.DataFrame:
    """
    배치 단위로 대화 데이터 생성
    
    Args:
        client: OpenAI 클라이언트
        batch_num: 배치 번호
        scenarios: 사용할 시나리오 리스트
        target_samples: 목표 샘플 수
        delay: API 호출 간 딜레이
    """
    print(f"\n{'='*80}")
    print(f"📦 배치 {batch_num} 생성 시작 (시나리오 {len(scenarios)}개 → 목표 {target_samples}개)")
    print(f"{'='*80}\n")
    
    conversations = []
    batch_start_tokens = token_tracker.total_tokens
    start_time = time.time()  # 시작 시간 기록
    
    # 각 시나리오당 생성할 샘플 수 계산
    samples_per_scenario = target_samples // len(scenarios)
    remaining_samples = target_samples % len(scenarios)
    
    for idx, scenario in enumerate(scenarios, 1):
        # 이 시나리오에서 생성할 샘플 수
        num_samples = samples_per_scenario
        if idx <= remaining_samples:
            num_samples += 1
        
        if num_samples == 0:
            continue
        
        scenario_success = 0
        scenario_fail = 0
        scenario_names = []  # 생성된 이름들 저장
        
        for sample_idx in range(num_samples):
            # 참여자 이름 생성 (2명)
            participant_names = name_generator.generate_names(2)
            
            # 대화 생성
            conversation = generate_single_conversation(
                client=client,
                scenario=scenario,
                participant_names=participant_names
            )
            
            if conversation:
                conversations.append(conversation)
                scenario_success += 1
                scenario_names.append(f"({', '.join(participant_names)})")
            else:
                scenario_fail += 1
            
            # API 호출 간 딜레이
            time.sleep(delay)
        
        # 시나리오별 결과를 한 줄로 출력 (이름 포함)
        status = "✅" if scenario_fail == 0 else "⚠️"
        names_str = " ".join(scenario_names) if scenario_names else ""
        print(f"{status} [{idx}/{len(scenarios)}] {scenario['type']}: {scenario_success}개 생성 {names_str}" + 
              (f" ({scenario_fail}개 실패)" if scenario_fail > 0 else ""))
        
        # 진행바 + 예상 시간 표시
        if idx % 5 == 0 or idx == len(scenarios):
            progress = idx / len(scenarios)
            bar_length = 40
            filled = int(bar_length * progress)
            bar = '█' * filled + '░' * (bar_length - filled)
            
            # 경과 시간 및 예상 남은 시간 계산
            elapsed_time = time.time() - start_time
            avg_time_per_scenario = elapsed_time / idx
            remaining_scenarios = len(scenarios) - idx
            estimated_remaining = avg_time_per_scenario * remaining_scenarios
            
            # 시간 포맷팅
            elapsed_str = f"{int(elapsed_time//60)}:{int(elapsed_time%60):02d}"
            if estimated_remaining > 0:
                remaining_str = f"{int(estimated_remaining//60)}:{int(estimated_remaining%60):02d}"
                time_info = f"[{elapsed_str}<{remaining_str}, {avg_time_per_scenario:.2f}s/it]"
            else:
                time_info = f"[{elapsed_str}, {avg_time_per_scenario:.2f}s/it]"
            
            print(f"📊 배치 {batch_num}: [{bar}] {progress*100:.1f}% ({idx}/{len(scenarios)}) {time_info} | 생성: {len(conversations)}개\n")
    
    # 배치 통계
    batch_tokens = token_tracker.total_tokens - batch_start_tokens
    token_tracker.add_batch_stats(batch_num, len(conversations), batch_tokens)
    
    total_time = time.time() - start_time
    print(f"⭐ 배치 {batch_num} 완료: {len(conversations)}/{target_samples}개 생성, 토큰: {batch_tokens:,}, 소요시간: {int(total_time//60)}분 {int(total_time%60)}초\n")
    
    # DataFrame 생성
    if conversations:
        df = pd.DataFrame(conversations)
        return df
    else:
        return pd.DataFrame()


print("✅ 배치 생성 함수 정의 완료")

✅ 배치 생성 함수 정의 완료


9. 데이터 저장 함수

In [10]:
def save_batch_data(df: pd.DataFrame, batch_num: int):
    """배치 데이터 저장 (CSV, JSON)"""
    
    # 기본 버전 (dialogue, label만)
    df_basic = df[['dialogue', 'label']].copy()
    basic_csv = os.path.join(OUTPUT_DIR, f"s1_batch_{batch_num:02d}_basic.csv")
    df_basic.to_csv(basic_csv, index=False, encoding='utf-8-sig')
    
    # 상세 버전 (모든 컬럼)
    detailed_csv = os.path.join(OUTPUT_DIR, f"s1_batch_{batch_num:02d}_detailed.csv")
    df.to_csv(detailed_csv, index=False, encoding='utf-8-sig')
    
    # JSON 버전
    json_file = os.path.join(OUTPUT_DIR, f"s1_batch_{batch_num:02d}.json")
    df.to_json(json_file, orient='records', force_ascii=False, indent=2)
    
    print(f"💾 배치 {batch_num} 저장 완료:")
    print(f"   - {basic_csv}")
    print(f"   - {detailed_csv}")
    print(f"   - {json_file}\n")


def save_all_data(all_dfs: List[pd.DataFrame]):
    """모든 배치 데이터 통합 저장"""
    
    # 모든 배치 통합
    df_all = pd.concat(all_dfs, ignore_index=True)
    
    # 기본 버전
    df_basic = df_all[['dialogue', 'label']].copy()
    basic_csv = os.path.join(OUTPUT_DIR, "s1_all_data_basic.csv")
    df_basic.to_csv(basic_csv, index=False, encoding='utf-8-sig')
    
    # 상세 버전
    detailed_csv = os.path.join(OUTPUT_DIR, "s1_all_data_detailed.csv")
    df_all.to_csv(detailed_csv, index=False, encoding='utf-8-sig')
    
    # JSON 버전
    json_file = os.path.join(OUTPUT_DIR, "s1_all_data.json")
    df_all.to_json(json_file, orient='records', force_ascii=False, indent=2)
    
    print(f"💾 통합 데이터 저장 완료:")
    print(f"   - {basic_csv}")
    print(f"   - {detailed_csv}")
    print(f"   - {json_file}\n")
    
    return df_all


print("✅ 데이터 저장 함수 정의 완료")

✅ 데이터 저장 함수 정의 완료


10. 메인 생성 함수

In [11]:
def main_generation():
    """메인 생성 프로세스"""
    
    print("\n" + "="*80)
    print("🚀 S1 스피어 피싱 데이터셋 생성 시작")
    print("="*80)
    print(f"총 시나리오: {len(ALL_SCENARIOS)}개")
    print(f"목표 샘플: {TOTAL_TARGET_SAMPLES}개")
    print(f"배치 수: {NUM_BATCHES}개 (각 {BATCH_SIZE}개)")
    print("="*80 + "\n")
    
    if not client:
        print("❌ 클라이언트가 초기화되지 않았습니다.")
        return
    
    all_dfs = []
    overall_start = time.time()
    
    # 시나리오를 배치별로 분할
    scenarios_per_batch = len(ALL_SCENARIOS) // NUM_BATCHES
    
    for batch_num in range(1, NUM_BATCHES + 1):
        # 이 배치에서 사용할 시나리오 선택
        start_idx = (batch_num - 1) * scenarios_per_batch
        if batch_num == NUM_BATCHES:
            # 마지막 배치는 남은 모든 시나리오 사용
            end_idx = len(ALL_SCENARIOS)
        else:
            end_idx = start_idx + scenarios_per_batch
        
        batch_scenarios = ALL_SCENARIOS[start_idx:end_idx]
        
        # 배치 생성
        df_batch = generate_batch(
            client=client,
            batch_num=batch_num,
            scenarios=batch_scenarios,
            target_samples=BATCH_SIZE,
            delay=API_DELAY_SECONDS
        )
        
        if not df_batch.empty:
            # 배치 데이터 저장
            save_batch_data(df_batch, batch_num)
            all_dfs.append(df_batch)
        
        print("="*80)
        print(f"✅ 배치 {batch_num}/{NUM_BATCHES} 완료")
        print("="*80 + "\n\n")
    
    # 모든 데이터 통합 저장
    if all_dfs:
        df_all = save_all_data(all_dfs)
        
        # 최종 통계
        print("="*80)
        print("📊 최종 통계")
        print("="*80)
        print(f"총 생성 데이터: {len(df_all)}개")
        
        # 대화 턴 수 통계
        df_all['turn_count'] = df_all['dialogue'].apply(lambda x: len(x.split('\n')))
        print(f"\n대화 턴 수:")
        print(f"  평균: {df_all['turn_count'].mean():.1f}")
        print(f"  범위: {df_all['turn_count'].min()} ~ {df_all['turn_count'].max()}")
        
        # 생성된 이름 수
        print(f"\n생성된 고유 이름 수: {name_generator.generation_count}개")
        print(f"사용된 고유 이름 수: {len(name_generator.used_names)}개")
        
        # 공격 결과 분석
        if 'meta' in df_all.columns:
            outcomes = df_all['meta'].apply(lambda x: x.get('attack_outcome', 'unknown') if isinstance(x, dict) else 'unknown')
            print(f"\n공격 결과:")
            print(f"  성공: {(outcomes == 'success').sum()}개 ({(outcomes == 'success').sum()/len(df_all)*100:.1f}%)")
            print(f"  실패: {(outcomes == 'failed').sum()}개 ({(outcomes == 'failed').sum()/len(df_all)*100:.1f}%)")
        
        # 토큰 사용량
        token_tracker.print_summary()
        
        # 총 소요 시간
        total_time = time.time() - overall_start
        print(f"\n⏱️  총 소요 시간: {total_time/60:.1f}분 ({total_time:.1f}초)")
        
        print("\n✅ 전체 생성 완료!")
        print("="*80)
    else:
        print("❌ 생성된 데이터가 없습니다.")


print("✅ 메인 생성 함수 정의 완료")

✅ 메인 생성 함수 정의 완료


11. 실행

In [12]:
# 실행
if __name__ == "__main__":
    main_generation()


🚀 S1 스피어 피싱 데이터셋 생성 시작
총 시나리오: 289개
목표 샘플: 20개
배치 수: 1개 (각 20개)


📦 배치 1 생성 시작 (시나리오 289개 → 목표 20개)

✅ [1/289] 급여 계좌 확인 요청: 1개 생성 (윤지후, 민예린)
✅ [2/289] 외부 법무 조사: 1개 생성 (황도윤, 허하은)
✅ [3/289] 수표/계좌 입금 확인: 1개 생성 (전연우, 허예준)
✅ [4/289] 패치·라이선스 확인: 1개 생성 (윤현우, 오유리)
✅ [5/289] 도메인/인증서 갱신: 1개 생성 (최지호, 최준호)
📊 배치 1: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 1.7% (5/289) [1:08<64:44, 13.68s/it] | 생성: 5개

✅ [6/289] 외부 결제 오류: 1개 생성 (노민재, 문서하)
✅ [7/289] 거래처 계좌 변경 요청: 1개 생성 (윤승호, 이서우)
✅ [8/289] 감사자료 제출: 1개 생성 (주하윤, 구연우)
✅ [9/289] 신원확인(법집행기관): 1개 생성 (남도훈, 김도훈)
✅ [10/289] 보험금/보상 처리: 1개 생성 (황지원, 성하민)
📊 배치 1: [█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 3.5% (10/289) [2:10<60:34, 13.03s/it] | 생성: 10개

✅ [11/289] 물류/택배 문제: 1개 생성 (문시현, 주지안)
✅ [12/289] 보안카드/출입증 재발급: 1개 생성 (심지안, 고현우)
✅ [13/289] 하드웨어 보수 요청: 1개 생성 (황민성, 심민재)
✅ [14/289] 계약서 전자서명 문제: 1개 생성 (정주원, 양채원)
✅ [15/289] 퇴직자 계정 처리: 1개 생성 (구현우, 한연우)
📊 배치 1: [██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 5.2% (15/289) [3:16<59:40, 13.07s/it] | 생성: 15개

✅ [16/289] 비